In [1]:
# Import required libraries
import pandas as pd
from dash import Dash, html, dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the SpaceX launch data into a pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")

# Create a dash application
app = Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                             options=[
                                                 {'label': 'All Sites', 'value': 'ALL'},
                                                 {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                 {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                                 {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                 {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'}
                                             ],
                                             value='ALL',
                                             placeholder="Select a Launch Site here",
                                             searchable=True
                                             ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the success rates for each site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                # TASK 3: Add a RangeSlider to select payload range
                                dcc.RangeSlider(
                                    id='payload-slider',
                                    min=int(spacex_df['Payload Mass (kg)'].min()),
                                    max=int(spacex_df['Payload Mass (kg)'].max()),
                                    step=1000,
                                    value=[int(spacex_df['Payload Mass (kg)'].min()), int(spacex_df['Payload Mass (kg)'].max())],
                                    marks={i: str(i) for i in range(0, int(spacex_df['Payload Mass (kg)'].max()) + 1, 1000)}
                                ),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    Output('success-pie-chart', 'figure'),
    [Input('site-dropdown', 'value')]
)
def render_pie_chart(selected_site):
    if selected_site == 'ALL':
        # Calculate success rates for all sites
        success_rates = []
        site_labels = []
        for site in spacex_df['Launch Site'].unique():
            filtered_df = spacex_df[spacex_df['Launch Site'] == site]
            success_count = len(filtered_df[filtered_df['class'] == 1])
            total_count = len(filtered_df)
            success_rate = success_count / total_count
            success_rates.append(success_rate)
            site_labels.append(site)

        # Create a pie chart for success rates at all sites
        fig = px.pie(names=site_labels, values=success_rates, title='Success Rates at All Sites')
    else:
        # Calculate success rates for the selected site
        filtered_df = spacex_df[spacex_df['Launch Site'] == selected_site]
        success_count = len(filtered_df[filtered_df['class'] == 1])
        failure_count = len(filtered_df[filtered_df['class'] == 0])
        fig = px.pie(names=['Success', 'Failure'], values=[success_count, failure_count],
                     title=f'Success vs. Failure for {selected_site}')

    return fig

# TASK 4: Add a callback function to render the success-payload-scatter-chart scatter plot
@app.callback(
    Output('success-payload-scatter-chart', 'figure'),
    [Input('site-dropdown', 'value'),
     Input('payload-slider', 'value')]
)
def render_scatter_chart(selected_site, payload_range):
    if selected_site == 'ALL':
        filtered_df = spacex_df[
            (spacex_df['Payload Mass (kg)'] >= payload_range[0]) & (spacex_df['Payload Mass (kg)'] <= payload_range[1])
        ]
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class',
                         color='Booster Version Category',
                         title='Payload vs. Launch Outcome (All Sites)')
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == selected_site]
        filtered_df = filtered_df[
            (filtered_df['Payload Mass (kg)'] >= payload_range[0]) & (filtered_df['Payload Mass (kg)'] <= payload_range[1])
        ]
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class',
                         color='Booster Version Category',
                         title=f'Payload vs. Launch Outcome for {selected_site}')

    return fig

# Run the app
if __name__ == '__main__':
    app.run_server()
